In [1]:
!free -mh

               total        used        free      shared  buff/cache   available
Mem:           503Gi       111Gi       280Gi       1.1Gi       110Gi       387Gi
Swap:             0B          0B          0B


# `STN(Spatial Transformer Networks) 공간 변형 네트워크`

----
[튜토리얼]

<설명> <br>
- Visual Attention 을 이용하여 데이터증강(augment)에서 이득을 본다.
<br>
- 미분 가능한 어텐션의 일반화이다.
<br>
- 이미지의 관심 영역을 자르고 ,크기 조정, 방향 수정이 가능하다.
<br>
- CNN 은 크기조정,회전 등에 민감하여 STN 이 극복에 유ㅜ용하다.
<br>
- 가장큰 장점으로 아주 작은 수정으로 기존의 CNN 과 간단하게 연결이 가능하다는 것이다.

----

In [2]:
# 라이센스: BSD
# 저자: Ghassen Hamrouni

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

plt.ion()   # 대화형 모드

# 1. Loading Data (데이터불러오기)

In [3]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')] # Mozilla 제가 옛날에 html , css , java-script 의 기초를 공부했던 사이트!!
urllib.request.install_opener(opener)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 학습용 데이터셋
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=64, shuffle=True, num_workers=4)
# 테스트용 데이터셋
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=64, shuffle=True, num_workers=4)

In [4]:
train_loader #train_loader 는 우리가 이미 만들었던대로 데이터가 정제되어 딥러닝에 들어가기 깔끔하게 만들어진다.


In [5]:
# train_loader안의 실제값 한개 뽑아서 확인 
dataiter = iter(train_loader)
images, labels = dataiter.__next__()
images.size()

torch.Size([64, 1, 28, 28])

```python
현재
train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=64, shuffle=True, num_workers=4
를 보면
train 데이터로 쓸 것이고 그냥 데이터 이미지를  tensor 로 바꾸고 정규화시키고 batch_size는 64로 만들겠다는 의미입니다.
```

# 2. STN 구성하기

`(1) localization network(위치결정네트워크) 구성` : 일반적인 CNN 의 역할. 공간 변환을 통해 파라미터를 예측하고 신경망이 자동 학습을 하여 데이터셋에서 명시되지는 않는다.

`(2) grid generator(그리드 생성기)` : 출력 이미지와 대응되는 각 픽설에 이미지 내 좌표 공간(grid)을 생성. 
`(3) sampler(샘플러)` : 공간 변환 파라미터를 입력 이미지에 적용

`-` affine_grid 및 grid_sample 모듈이 포함된 최신 버전의 PyTorch가 필요합니다.

`(3) 함수보기전 데이터 미리보기` : <br>
    `-` MNIST 데이터이므로 흑백사진이라서 1*28*28 의 이미지라는 것을 알 수 있다. <br>
    `-` train_loader 을 통해 transform 을 지나가며 batch_size = 64 이고 (평균: 0.13 , 분산 : 0.3 으로 정규화진행) (batch_size, 1, 28, 28) 인 tensor 가 나오게됩니다.

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #채널: 입력 1 , 출력10 , 5*5 필터사용
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

        # 공간 변환을 위한 위치 결정 네트워크 (localization-network)
        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        ) # tensor([64,10,3,3]) 그래서 밑에보면 10*3*3 을 넣는다.

        # [3 * 2] 크기의 아핀(affine) 행렬에 대해 예측
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 3 * 3, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )

        # 항등 변환(identity transformation)으로 가중치/바이어스 초기화
        self.fc_loc[2].weight.data.zero_() #3번째 layer 가중치 0 만들기
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    # STN의 forward 함수
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3) #미니배치크기에 맞춘다는 의미
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x

    def forward(self, x):
        # 입력을 변환
        x = self.stn(x)

        # 일반적인 forward pass를 수행
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x)) #50개로 변환
        x = F.dropout(x, training=self.training)
        x = self.fc2(x) #10개로 변환 
        return F.log_softmax(x, dim=1) #10개를 log softmax 실시


model = Net().to(device)

[해석] <br>
`-` 첫 번째 컨볼루션 레이어 (nn.Conv2d(1, 8, kernel_size=7)):
<br>
- 입력 채널: 1 (흑백 이미지이므로)
- 출력 채널: 8
- 커널 크기: 7x7
- 출력 크기: (28 - 7 + 2 * 0) + 1 = 22
> 따라서 출력 텐서 크기는 (64, 8, 22, 22)가 됩니다.

-----

`-` 첫 번째 맥스 풀링 레이어 (nn.MaxPool2d(2, stride=2)):

- 풀링 윈도우 크기: 2x2
- 스트라이드: 2
- 출력 크기: (22 - 2) / 2 + 1 = 11
> 따라서 출력 텐서 크기는 (64, 8, 11, 11)이 됩니다.

-----

`-` **ReLU** 활성화 함수 (nn.ReLU(True)):
<br>
활성화 함수를 통과하므로 텐서 크기는 변하지 않습니다.

`-` 두 번째 컨볼루션 레이어 (nn.Conv2d(8, 10, kernel_size=5)):
<br>
- 입력 채널: 8
- 출력 채널: 10
- 커널 크기: 5x5
- 출력 크기: (11 - 5 + 2 * 0) + 1 = 7
> 따라서 출력 텐서 크기는 (64, 10, 7, 7)이 됩니다.

`-` 두 번째 맥스 풀링 레이어 (nn.MaxPool2d(2, stride=2)):
<br>
- 풀링 윈도우 크기: 2x2
- 스트라이드: 2
- 출력 크기: (7 - 2) / 2 + 1 = 3
> 따라서 출력 텐서 크기는 (64, 10, 3, 3)이 됩니다.

> 따라서 `self.localization`에 의해 처리된 후의 텐서 크기는 (64, 10, 3, 3)이 됩니다

```python
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3) #미니배치크기에 맞춘다는 의미
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x
```
<br>
1.
xs = self.localization(x): Localization Network를 통과한 결과인 xs는 크기가 (64, 10, 3, 3)인 텐서입니다.
<br>
2.
xs = xs.view(-1, 10 * 3 * 3): xs를 2D 텐서로 평탄화합니다. 결과적으로 xs는 크기가 (64, 90)인 텐서로 변합니다.
<br>
3.
theta = self.fc_loc(xs): 평탄화된 xs를 Fully Connected 레이어에 통과시켜 공간 변환 매개변수 theta를 얻습니다. theta의 크기는 (64, 6)입니다. #fc_loc 라는 함수를 linear(90,32) -> linear(32,6) 을 진행했었다.
<br>
4.
theta = theta.view(-1, 2, 3): theta를 크기가 (64, 2, 3)인 3D 텐서로 변환합니다. 이는 공간 변환 매개변수를 나타냅니다.
<br>
5.
grid = F.affine_grid(theta, x.size()): Affine 그리드를 계산합니다. 이 그리드는 입력 이미지에 적용할 공간적인 변환을 정의합니다.
<br>
`-` affine_grid : x.size() 의 크기로 이미지를 공간적 변환을 선현 변환과 평행 이동으로 만드는 함수입니다.
<br>
6.
x = F.grid_sample(x, grid): Affine 그리드를 사용하여 입력 이미지 x에 공간적인 변환을 적용합니다.


## `최종적인 x의 크기는 (64, 1, 28, 28)로 유지됩니다.`

# forward(x)

> 이제 forward 로 input, output 실행하기

`-` return F.log_softmax(x, dim=1) : x 가 결국10개의 output 층으로 나타나고 이 후 log softmax 에 의해 10가지를 dim=1 인 각 클래스에 속할 확률의 로그값의 텐서 10개로 반환합니다.

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
#
# MNIST 데이터셋에서 STN의 성능을 측정하기 위한 간단한 테스트 절차
#


def test():
    with torch.no_grad():
        model.eval() #pytorch 에서 모델 평가하는 메서드이다.
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # 배치 손실 합하기
            test_loss += F.nll_loss(output, target, size_average=False).item()
            # 로그-확률의 최대값에 해당하는 인덱스 가져오기
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct / len(test_loader.dataset)))

# STN 결과 시각화하기
`-` 매커니즘의 결과를 보고 시각화에 도움되는 함수 만들기

In [8]:
def convert_image_np(inp):
    """Convert a Tensor to numpy image."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp

# 학습 후 공간 변환 계층의 출력을 시각화하고, 입력 이미지 배치 데이터 및
# STN을 사용해 변환된 배치 데이터를 시각화 합니다.


def visualize_stn():
    with torch.no_grad():
        # 학습 데이터의 배치 가져오기
        data = next(iter(test_loader))[0].to(device)

        input_tensor = data.cpu()
        transformed_input_tensor = model.stn(data).cpu()

        in_grid = convert_image_np(
            torchvision.utils.make_grid(input_tensor))

        out_grid = convert_image_np(
            torchvision.utils.make_grid(transformed_input_tensor))

        # 결과를 나란히 표시하기
        f, axarr = plt.subplots(1, 2)
        axarr[0].imshow(in_grid)
        axarr[0].set_title('Dataset Images')

        axarr[1].imshow(out_grid)
        axarr[1].set_title('Transformed Images')



In [ ]:
for epoch in range(1, 20 + 1):
    train(epoch)
    test()

# 일부 입력 배치 데이터에서 STN 변환 결과를 시각화
visualize_stn()

plt.ioff()
plt.show()